<a href="https://colab.research.google.com/github/Parv17k/nyc_collision_data_analysis/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [39]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [35]:
!wget -O longlat.csv -q https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

In [40]:
ls

longlat.csv  -q  sample_data/  spark-3.1.1-bin-hadoop2.7.tgz


In [41]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [42]:
!pip install -q findspark

In [43]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [44]:
import findspark
findspark.init()

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-27 21:10:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.209.152.48, 3.225.89.236, 54.235.108.207, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.209.152.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13805791 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.17M  42.5MB/s    in 0.3s    

2021-03-27 21:10:07 (42.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13805791/13805791]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -q https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

In [ ]:
mv rows.csv col.csv


In [ ]:
ls


 col.csv                          ngrok-stable-linux-amd64.zip.2
 drive/                          'rows.csv?accessType=DOWNLOAD'
 ngrok*                           sample_data/
 ngrok-stable-linux-amd64.zip     spark-3.1.1-bin-hadoop2.7/
 ngrok-stable-linux-amd64.zip.1   spark.tgz


In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col


In [ ]:
spark =SparkSession(sc)

In [ ]:
col=spark.read.option("header",True).csv("./col.csv",inferSchema=True)
longlat=spark.read.option("delimiter",";").csv("./longlat.csv",inferSchema=True,header=True)

In [ ]:
col.count()-col.distinct().count()

0

In [ ]:
len(col.columns)


29

In [ ]:
col.printSchema()

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [ ]:
import seaborn as sns

In [ ]:
#dropping columns with collision id = null

In [ ]:
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[CRASH DATE: string, CRASH TIME: string, BOROUGH: string, ZIP CODE: string, LATITUDE: double, LONGITUDE: double, LOCATION: string, ON STREET NAME: string, CROSS STREET NAME: string, OFF STREET NAME: string, NUMBER OF PERSONS INJURED: string, NUMBER OF PERSONS KILLED: int, NUMBER OF PEDESTRIANS INJURED: int, NUMBER OF PEDESTRIANS KILLED: int, NUMBER OF CYCLIST INJURED: int, NUMBER OF CYCLIST KILLED: string, NUMBER OF MOTORIST INJURED: string, NUMBER OF MOTORIST KILLED: int, CONTRIBUTING FACTOR VEHICLE 1: string, CONTRIBUTING FACTOR VEHICLE 2: string, CONTRIBUTING FACTOR VEHICLE 3: string, CONTRIBUTING FACTOR VEHICLE 4: string, CONTRIBUTING FACTOR VEHICLE 5: string, COLLISION_ID: int, VEHICLE TYPE CODE 1: string, VEHICLE TYPE CODE 2: string, VEHICLE TYPE CODE 3: string, VEHICLE TYPE CODE 4: string, VEHICLE TYPE CODE 5: string]

In [ ]:
col.count()

1764791

In [ ]:
missingLocation=col.filter(col["BOROUGH"].isNull()).filter(col["ZIP CODE"].isNull())

In [ ]:
cannotRetriveLocation =missingLocation.filter(col["LONGITUDE"].isNull())

In [ ]:
cannotRetriveLocation.count()

177335

In [ ]:
# Infer the schema, and register the DataFrame as a table.

col.createOrReplaceTempView("col");
cannotRetriveLocation.createOrReplaceTempView("toDelete")

In [ ]:
colNew= spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null ");


In [ ]:
colNew.count()

1587454

In [ ]:
col.count() - cannotRetriveLocation.count()

1587456

In [ ]:
colNew.filter(colNew["ZIP CODE"].isNull()).filter(colNew["BOROUGH"].isNotNull()).count()

211

In [ ]:
BOROUGH=list(map(lambda x : x["BOROUGH"],colNew.select("BOROUGH").distinct().collect()))

In [ ]:
BOROUGH

[None, 'QUEENS', 'BROOKLYN', 'BRONX', 'MANHATTAN', 'STATEN ISLAND']

In [ ]:
longlat.printSchema()

root
 |-- zip: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- geopoint: string (nullable = true)



In [ ]:

colNew.createOrReplaceTempView("colNew");
longlat.createOrReplaceTempView("longlat");

In [ ]:
missingZipCode=colNew.filter(colNew['ZIP CODE'].isNull());


+--------+
|count(1)|
+--------+
|  365086|
+--------+



In [ ]:
missingZipCode.createOrReplaceTempView("missingZipCode");


365086

In [ ]:
for m in missingZipCode.limit(19).collect():
  zip=spark.sql("select zip,city FROM longlat where ROUND(longlat.latitude,2) ="+str(round(m.LATITUDE,2))+" and ROUND(longlat.longitude,2)= "+str(round(m.LONGITUDE,2))+" limit 1");
  zip=zip.select("zip","city").collect()
  if len(zip)>0:
    if zip[0].city.upper()=="NEW YORK":
      print(m["COLLISION_ID"],"MANHATTAN",zip[0].zip)
    else:
        if zip[0].city.upper() in BOROUGH:
          print(m["COLLISION_ID"],zip[0].city.upper(),zip[0].zip)

4351127 MANHATTAN 10115
4398894 MANHATTAN 10019
4401423 BROOKLYN 11238
4297036 BROOKLYN 11233
4348050 MANHATTAN 10025
4397532 MANHATTAN 10039
4395839 BROOKLYN 11237
